In [1]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os

In [2]:
from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP

bulkDataPath = os.path.join('D:\\', 'CMPT_489_Bulk', 'Data')
print(bulkDataPath)
print("Path exists:", os.path.isdir(bulkDataPath))

from tqdm.notebook import tqdm_notebook

C:\Users\llbue\anaconda3\envs\DeepLearning\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


D:\CMPT_489_Bulk\Data
Path exists: True


In [3]:
# ------- 1. define loss function --------

bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = bce_loss(d0,labels_v)
	loss1 = bce_loss(d1,labels_v)
	loss2 = bce_loss(d2,labels_v)
	loss3 = bce_loss(d3,labels_v)
	loss4 = bce_loss(d4,labels_v)
	loss5 = bce_loss(d5,labels_v)
	loss6 = bce_loss(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	#print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0.float(), loss.float()

C:\Users\llbue\anaconda3\envs\DeepLearning\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [4]:
# ------- 2. set the directory of training dataset --------

model_name = 'u2netp_sub_vizwiz' #'u2netp'

data_dir = os.path.join(bulkDataPath + os.sep)
tra_image_dir = os.path.join('sub_matching_data', 'train' + os.sep)
tra_label_dir = os.path.join('sub_binary_masks', 'train' + os.sep)

image_ext = '.png'
label_ext = '.png'

model_dir = os.path.join(os.getcwd(), 'saved_models', model_name + os.sep)

epoch_num = 50 #100000
batch_size_train = 16 #12
batch_size_val = 1
train_num = 0
val_num = 0

tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)

tra_lbl_name_list = []
for img_path in tra_img_name_list:
	img_name = img_path.split(os.sep)[-1]

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	tra_lbl_name_list.append(data_dir + tra_label_dir + imidx + label_ext)

print("---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("---")

print(tra_img_name_list[0:3])
print(tra_lbl_name_list[0:3])

---
train images:  7500
train labels:  7500
---
['D:\\CMPT_489_Bulk\\Data\\sub_matching_data\\train\\VizWiz_train_00000000.png', 'D:\\CMPT_489_Bulk\\Data\\sub_matching_data\\train\\VizWiz_train_00000002.png', 'D:\\CMPT_489_Bulk\\Data\\sub_matching_data\\train\\VizWiz_train_00000004.png']
['D:\\CMPT_489_Bulk\\Data\\sub_binary_masks\\train\\VizWiz_train_00000000.png', 'D:\\CMPT_489_Bulk\\Data\\sub_binary_masks\\train\\VizWiz_train_00000002.png', 'D:\\CMPT_489_Bulk\\Data\\sub_binary_masks\\train\\VizWiz_train_00000004.png']


In [5]:
train_num = len(tra_img_name_list)

salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))
salobj_dataloader = DataLoader(salobj_dataset, batch_size=batch_size_train, shuffle=True, num_workers=1)

In [6]:
# ------- 3. define model --------
# define the net
if(model_name.split('_')[0]=='u2net'):
    net = U2NET(3, 1)
    print('u2net')
elif(model_name.split('_')[0]=='u2netp'):
    net = U2NETP(3,1)
    print('u2netp')

if torch.cuda.is_available():
    net.cuda()
    print("cuda")

u2netp
cuda


In [7]:
# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

---define optimizer...


In [8]:
# ------- 5. training process --------
print("---start training...")
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 1000 # save the model every 2000 iterations

for epoch in range(0, epoch_num):
    net.train()
    
    for i, data in enumerate(tqdm_notebook(salobj_dataloader)):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)
        
        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)
        # y zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(inputs_v.float())
        loss2, loss = muti_bce_loss_fusion(d0.float(), d1.float(), d2.float(), d3.float(), d4.float(), d5.float(), d6.float(), labels_v.float())
        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data.item()
        running_tar_loss += loss2.data.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        if ite_num % 50 == 0:
            print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
            epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % save_frq == 0:

            torch.save(net.state_dict(), model_dir + model_name+"_bce_itr_%d_train_%3f_tar_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0

---start training...


  0%|          | 0/469 [00:00<?, ?it/s]

C:\Users\llbue\anaconda3\envs\DeepLearning\lib\site-packages\torch\nn\functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
C:\Users\llbue\anaconda3\envs\DeepLearning\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch:   1/ 50, batch:   800/ 7500, ite: 50] train loss: 4.590174, tar: 0.657336 
[epoch:   1/ 50, batch:  1600/ 7500, ite: 100] train loss: 4.496418, tar: 0.647364 
[epoch:   1/ 50, batch:  2400/ 7500, ite: 150] train loss: 4.423298, tar: 0.635480 
[epoch:   1/ 50, batch:  3200/ 7500, ite: 200] train loss: 4.382692, tar: 0.628119 
[epoch:   1/ 50, batch:  4000/ 7500, ite: 250] train loss: 4.350256, tar: 0.622678 
[epoch:   1/ 50, batch:  4800/ 7500, ite: 300] train loss: 4.309331, tar: 0.616072 
[epoch:   1/ 50, batch:  5600/ 7500, ite: 350] train loss: 4.279991, tar: 0.611560 
[epoch:   1/ 50, batch:  6400/ 7500, ite: 400] train loss: 4.256153, tar: 0.607718 
[epoch:   1/ 50, batch:  7200/ 7500, ite: 450] train loss: 4.237548, tar: 0.604875 


  0%|          | 0/469 [00:00<?, ?it/s]

...

[epoch:  48/ 50, batch:   112/ 7500, ite: 22050] train loss: 1.458662, tar: 0.196529 
[epoch:  48/ 50, batch:   912/ 7500, ite: 22100] train loss: 1.470723, tar: 0.198307 
[epoch:  48/ 50, batch:  1712/ 7500, ite: 22150] train loss: 1.449739, tar: 0.195515 
[epoch:  48/ 50, batch:  2512/ 7500, ite: 22200] train loss: 1.440297, tar: 0.194257 
[epoch:  48/ 50, batch:  3312/ 7500, ite: 22250] train loss: 1.455241, tar: 0.196520 
[epoch:  48/ 50, batch:  4112/ 7500, ite: 22300] train loss: 1.464627, tar: 0.197813 
[epoch:  48/ 50, batch:  4912/ 7500, ite: 22350] train loss: 1.444799, tar: 0.194998 
[epoch:  48/ 50, batch:  5712/ 7500, ite: 22400] train loss: 1.463236, tar: 0.197637 
[epoch:  48/ 50, batch:  6512/ 7500, ite: 22450] train loss: 1.457702, tar: 0.196791 
[epoch:  48/ 50, batch:  7312/ 7500, ite: 22500] train loss: 1.464141, tar: 0.197723 


  0%|          | 0/469 [00:00<?, ?it/s]

[epoch:  49/ 50, batch:   608/ 7500, ite: 22550] train loss: 1.462907, tar: 0.197505 
[epoch:  49/ 50, batch:  1408/ 7500, ite: 22600] train loss: 1.458185, tar: 0.196735 
[epoch:  49/ 50, batch:  2208/ 7500, ite: 22650] train loss: 1.458473, tar: 0.196778 
[epoch:  49/ 50, batch:  3008/ 7500, ite: 22700] train loss: 1.449434, tar: 0.195536 
[epoch:  49/ 50, batch:  3808/ 7500, ite: 22750] train loss: 1.442729, tar: 0.194596 
[epoch:  49/ 50, batch:  4608/ 7500, ite: 22800] train loss: 1.437421, tar: 0.193796 
[epoch:  49/ 50, batch:  5408/ 7500, ite: 22850] train loss: 1.442085, tar: 0.194502 
[epoch:  49/ 50, batch:  6208/ 7500, ite: 22900] train loss: 1.437622, tar: 0.193872 
[epoch:  49/ 50, batch:  7008/ 7500, ite: 22950] train loss: 1.443002, tar: 0.194614 


  0%|          | 0/469 [00:00<?, ?it/s]

[epoch:  50/ 50, batch:   304/ 7500, ite: 23000] train loss: 1.446585, tar: 0.195125 
[epoch:  50/ 50, batch:  1104/ 7500, ite: 23050] train loss: 1.373618, tar: 0.182556 
[epoch:  50/ 50, batch:  1904/ 7500, ite: 23100] train loss: 1.464889, tar: 0.196518 
[epoch:  50/ 50, batch:  2704/ 7500, ite: 23150] train loss: 1.465368, tar: 0.197254 
[epoch:  50/ 50, batch:  3504/ 7500, ite: 23200] train loss: 1.429059, tar: 0.192067 
[epoch:  50/ 50, batch:  4304/ 7500, ite: 23250] train loss: 1.421225, tar: 0.190837 
[epoch:  50/ 50, batch:  5104/ 7500, ite: 23300] train loss: 1.421384, tar: 0.190714 
[epoch:  50/ 50, batch:  5904/ 7500, ite: 23350] train loss: 1.428863, tar: 0.191794 
[epoch:  50/ 50, batch:  6704/ 7500, ite: 23400] train loss: 1.420352, tar: 0.190700 
[epoch:  50/ 50, batch:  7504/ 7500, ite: 23450] train loss: 1.432142, tar: 0.192547 
